In [1]:
#!/usr/bin/env python

## time stamp corrector

Authors: L. Combi & F. García

Date: 11/02/19

This script corrects the time stamp of a raw Filterbank observation.

In [ ]:
import os, sys
from sigproc import *
import numpy as np
from astropy.Time import Time
import glob

In [ ]:
# Make a list of all availabe Filterbank files in the folder

fils = glob.glob('*.fil')

In [ ]:
# Make a dic of all the starting time-stamps (ts)

tsold = {}

for x in fils:
    
       tsdic[x] = read_header(x)[0]['tstart']

In [ ]:
# We transform the ts from MJDs to isot format, we correct the non zero values and we re reconvert.
# Save them in a new dictionary.

tsnew= {}

for i in range(len(fils)):
        tmp= Time(tsdic[fils[i]], format='mjd', scale='utc', precision=9)
        newisot = tmp.isot[:18]+str(round(float(tmp.isot[18:])))
        tmpmjd= Time(newisot, format='isot', scale='utc', precision=9)
        trdic[fils[i]] = tmpmjd.mjd

In [ ]:
for i in range(len(fils)):
    
    f = open(fils[i], 'rb')
    
    filhdr = {}
    
    newhdr = ""
    
    outfile = open(fils[i]+'tscorrected','wb')
    
    while 1:                                                              
               
            param, val = read_hdr_val(f, stdout=True)                        
                
            filhdr[param] = val                                     
                                                                           
             # Add lines here to correct timestamp
            if param=="tstart":  val = trdic[fils[i]]              
     
             # Append to the new hdr string
            newhdr += addto_hdr(param, val)
     
             # Break out of the loop if the header is over
            if param=="HEADER_END":  break
    
    outfile.write(newhdr)
    
    hdrlen = f.tell()
    
    numbytes = os.stat(fils[i])[6] - hdrlen
    
    m=0
    
    while m < numbytes:
        
        towrite = f.read(100000)
        m += 100000
        outfile.write(towrite)
    
    towrite = f.read(numbytes-f.tell())
    
    outfile.write(towrite)
    
    outfile.close()   
    